In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn.model_selection import train_test_split
from shutil import copyfile

mainDataDir='/kaggle/input/multi-cancer/Multi Cancer/Cervical Cancer'
cervixDykDir=os.path.join(mainDataDir,'cervix_dyk')
cervixKocDir=os.path.join(mainDataDir,'cervix_koc')
cervixMepDir=os.path.join(mainDataDir,'cervix_mep')
cervixPabDir=os.path.join(mainDataDir,'cervix_pab')
cervixSfiDir=os.path.join(mainDataDir,'cervix_sfi')

In [3]:
trainDir = '/kaggle/working/training_data'
valDir = '/kaggle/working/validation_data'

In [4]:
os.makedirs(trainDir,exist_ok=True)
os.makedirs(valDir,exist_ok=True)

In [5]:
def split_and_copy(class_dir, train_output_dir, val_output_dir):
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('jpg', 'jpeg', 'png'))]

    train_images, val_images = train_test_split(images, test_size=0.3, random_state=42)

    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(val_output_dir, exist_ok=True)

    for img in train_images:
        copyfile(img, os.path.join(train_output_dir, os.path.basename(img)))

    for img in val_images:
        copyfile(img, os.path.join(val_output_dir, os.path.basename(img)))

split_and_copy(cervixDykDir, os.path.join(trainDir, 'cervix_dyk'), os.path.join(valDir, 'cervix_dyk'))
split_and_copy(cervixKocDir, os.path.join(trainDir, 'cervix_koc'), os.path.join(valDir, 'cervix_koc'))
split_and_copy(cervixMepDir, os.path.join(trainDir, 'cervix_mep'), os.path.join(valDir, 'cervix_mep'))
split_and_copy(cervixPabDir, os.path.join(trainDir, 'cervix_pab'), os.path.join(valDir, 'cervix_pab'))
split_and_copy(cervixSfiDir, os.path.join(trainDir, 'cervix_sfi'), os.path.join(valDir, 'cervix_sfi'))

## Data Augmentation

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    '/kaggle/working/training_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
)

val_generator=val_datagen.flow_from_directory(
    '/kaggle/working/validation_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
)

Found 17500 images belonging to 5 classes.
Found 7500 images belonging to 5 classes.


## Training the Data

In [7]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Input,InputLayer,GlobalAveragePooling1D,BatchNormalization

In [8]:
numClasses=len(train_generator.class_indices)

In [9]:
imageShape=(224,224,3)

In [10]:
from tensorflow.keras.regularizers import l2

model=Sequential([
    Input(shape=imageShape),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(numClasses,activation='softmax')
])

In [11]:
from keras.optimizers import Adam,SGD

In [12]:
optimizer = Adam(learning_rate=0.001) 
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,789,061 (10.64 MB)

 Trainable params: 2,789,061 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
es=EarlyStopping( monitor="val_loss", patience=3,
                                     verbose=1,  restore_best_weights=True)
rlronp=ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=1,
                                             verbose=1)
callbacks=[es, rlronp]

In [16]:
batch_size=16
steps_per_epoch = len(train_generator) // batch_size  
validation_steps = len(val_generator) // batch_size  
history = model.fit(train_generator,validation_data=val_generator,epochs=50,steps_per_epoch=steps_per_epoch,validation_steps=validation_steps,callbacks=callbacks)

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


34/34 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.2704 - loss: 1.7496 - val_accuracy: 0.4085 - val_loss: 1.2934 - learning_rate: 0.0010
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.4610 - loss: 1.2868 - val_accuracy: 0.5290 - val_loss: 1.2044 - learning_rate: 0.0010
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5345 - loss: 1.1376 - val_accuracy: 0.5223 - val_loss: 1.0902 - learning_rate: 0.0010
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5932 - loss: 1.0952 - val_accuracy: 0.6629 - val_loss: 0.8976 - learning_rate: 0.0010
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.6338 - loss: 0.8933 - val_accuracy: 0.7143 - val_loss: 0.7887 - learning_rate: 0.0010
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6053 - loss: 1.0392
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
34/34 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.6048 - loss: 1.0400 - val_accuracy: 0.7054 - val_

In [17]:
# Evaluate on the training generator
train_results = model.evaluate(train_generator,batch_size=64)
print("Train loss, Train accuracy:", train_results)

# Evaluate on the validation generator
val_results = model.evaluate(val_generator)
print("Validation loss, Validation accuracy:", val_results)

547/547 ━━━━━━━━━━━━━━━━━━━━ 389s 712ms/step - accuracy: 0.7521 - loss: 0.7312
Train loss, Train accuracy: [0.7332617044448853, 0.7455999851226807]
235/235 ━━━━━━━━━━━━━━━━━━━━ 84s 358ms/step - accuracy: 0.7605 - loss: 0.6882
Validation loss, Validation accuracy: [0.6832325458526611, 0.7635999917984009]


In [19]:
def savingModel(model, model_name="model"): 
    model_filename = f"{model_name}.h5"
    model.save(model_filename)
    print(f"Model saved as {model_filename}")

In [20]:
savingModel(model,model_name="CervicalCancer")

Model saved as CervicalCancer.h5


In [21]:
from keras.models import load_model

In [22]:
def loadingModel(model_name="model"): 
    try:
        model = load_model(f"{model_name}.h5")
        model.summary()  # Print model summary if needed
        return model  # Return the loaded model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [23]:
loadedModel=loadingModel("CervicalCancer")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,789,063 (10.64 MB)

 Trainable params: 2,789,061 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [24]:
def loadImg(imgPath): 
    img = tf.keras.preprocessing.image.load_img(imgPath, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array=img_array/255.0
    return img_array

In [25]:
import tensorflow as tf

In [26]:
def predictImagesInFolder(testFolder, class_name, model_name): 
    savedModel = loadingModel(model_name)
    print(savedModel)

    if savedModel is None:
        print("Model loading failed.")
        return

    for subdir, dirs, files in os.walk(testFolder): 
        if subdir == testFolder: 
            continue
        trueLabel = os.path.basename(subdir)
        for file in files: 
            img_path = os.path.join(subdir, file)
            img = loadImg(img_path)
            
            # Debugging prediction probabilities
            predictions = savedModel.predict(img)
#             print(f"Prediction Probabilities for {file}: {predictions}")

            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class = class_name[predicted_class_index[0]]
            print(f"Predicted class: {predicted_class} File: {file}")

In [27]:
def testFolder(folder): 
    testFolder=f"{folder}"
    return testFolder

In [28]:
CervicalCancerTestFolder=testFolder("/kaggle/working/validation_data")

In [29]:
train_generator.class_indices

{'cervix_dyk': 0,
 'cervix_koc': 1,
 'cervix_mep': 2,
 'cervix_pab': 3,
 'cervix_sfi': 4}

In [30]:
CervicalCancerClassname={0:'cervix_dyk', 1:'cervix_koc', 2:'cervix_mep', 3:'cervix_pab',4:'cervix_sfi'}

In [ ]:
predictImagesInFolder(CervicalCancerTestFolder,CervicalCancerClassname,model_name="CervicalCancer")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,789,063 (10.64 MB)

 Trainable params: 2,789,061 (10.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

<Sequential name=sequential, built=True>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Predicted class: cervix_dyk File: cervix_dyk_4728.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: cervix_dyk File: cervix_dyk_4824.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted class: cervix_koc File: cervix_dyk_2537.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: cervix_koc File: cervix_dyk_2747.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: cervix_dyk File: cervix_dyk_2945.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: cervix_dyk File: cervix_dyk_3962.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: cervix_dyk File: cervix_dyk_4691.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: cervix_dyk File: cervix_dyk_3062.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: cervix_dyk File: cervix_dyk_4284.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: cervix_dyk File: cervix_dyk_4404.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
